## Predicting Atomic Polarizability Using Machine Learning

 The formula for atomic polarizability is $\alpha = \dfrac{p}{E}$ where p is the dipole moment and E is the electric field.  
 P has units of Couloub * Distance = Cm
 E is measured in units of Netwons per coulomb = N/C  
 Based on the above, the formula for polarizability in proper units is $\alpha = \dfrac{Cm}{\dfrac{N}{C}}=\dfrac{C^2m}{N}$=$C^2mN−1$

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('Molecules.csv')
df.head()

,row ID,Molecule,Molecule name,Mannhold LogP,Atomic Polarizabilities,Aromatic Atoms Count,Aromatic Bonds Count,Element Count,Bond Polarizabilities,Bond Count,...,XLogP,Zagreb Index,Molecular Formula,Formal Charge,Formal Charge (pos),Formal Charge (neg),Heavy Atoms Count,Molar Mass,SP3 Character,Rotatable Bonds Count (non terminal)
0,Row0,N,H3N,1.35,3.100379,0,0,4,1.299621,0,...,0.000,0,H3N,0,0,0,1,17.0305254685134,0.000000,0
1,Row1,O=C(O)c1c(OC(=O)C)cccc1,C9H8O4,2.01,24.382344,6,6,21,12.577656,13,...,1.422,60,C9H8O4,0,0,0,13,180.157768811976,0.047619,3
2,Row2,C#C,C2H2,1.68,4.853586,0,0,4,2.186414,1,...,0.418,2,C2H2,0,0,0,2,26.0373533012276,0.000000,0
3,Row3,P(=O)(OP(=O)(OC[C@H]1O[C@@H](n2c3ncnc(c3nc2)N)...,C10H16N5O13P3,0.25,55.084688,9,10,47,55.749312,33,...,-5.533,170,C10H16N5O13P3,0,0,0,31,507.181476012675,0.106383,8
4,Row4,S1C([C@H](C(=O)O)N2[C@H]1[C@H](NC(=O)[C@H](N)c...,C16H19N3O4S,2.34,50.237067,6,6,43,27.904933,26,...,-1.732,132,C16H19N3O4S,0,0,0,24,349.406462995722,0.162791,4


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15166 entries, 0 to 15165
Data columns (total 32 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   row ID                                  15166 non-null  object 
 1   Molecule                                15166 non-null  object 
 2   Molecule name                           15166 non-null  object 
 3   Mannhold LogP                           15166 non-null  float64
 4   Aromatic Atoms Count                    15166 non-null  int64  
 5   Aromatic Bonds Count                    15166 non-null  int64  
 6   Element Count                           15166 non-null  int64  
 7   Bond Polarizabilities                   15166 non-null  object 
 8   Bond Count                              15166 non-null  int64  
 9   Eccentric Connectivity Index            15166 non-null  int64  
 10  Fragment Complexity                     15166 non-null  fl

In [3]:
# setting Atomic Polarizabilites as the target column
target = df['Atomic Polarizabilities']
del df['Atomic Polarizabilities']
target.head()

0     3.100379
1    24.382344
2     4.853586
3    55.084688
4    50.237067
Name: Atomic Polarizabilities, dtype: object